In [1]:
import pandas as pd

with open("data.txt", "r") as f:
    lines = f.readlines()
data = [line.strip().split(",") for line in lines][:-1] 
df = pd.DataFrame(data, columns=["s_len", "s_wid", "p_len", "p_wid", "class"])

label_mapping = {
    "Iris-setosa": 0,
    "Iris-versicolor": 1, # colour hahah
    "Iris-virginica": 2
}

df['class'] = df['class'].map(label_mapping)
df = df.astype(float)
df

,s_len,s_wid,p_len,p_wid,class
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [2]:
import numpy as np

In [3]:
def relu(x):
    return np.clip(x, 0, np.abs(np.max(x)))

In [4]:
relu(np.random.normal(size=(3,3)))

array([[0.        , 0.        , 0.        ],
       [0.11325282, 1.38680225, 0.        ],
       [0.14184724, 0.        , 0.        ]])

In [35]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [40]:
def make_one_hot(label):
    one_hot = np.zeros(3)
    one_hot[int(label)] = 1
    return one_hot

In [49]:
def cross_entropy_loss(y, y_pred):
    return np.mean(-np.sum(y * np.log(y_pred), axis=-1))

# full oop

In [5]:
class Neuron:
    def __init__(self, weight, activation_func):
        self.weight = weight
        self.activation_func = activation_func
    
    def __call__(self, data):
        h = np.sum(data) * self.weight
        if self.activation_func:
            h = self.activation_func(h)
        self.h = h
        return self.h

In [6]:
n = Neuron(-.25, relu)
n(df.iloc[0, :-1])

np.float64(0.0)

In [7]:
def get_random_weight_or_bias():
    return np.random.normal() - 0.5

class Layer:
    def __init__(self, hidden_size):
        self.hidden_size = hidden_size
    
    def initialize(self):
        self.neurons = [Neuron(get_random_weight_or_bias(), relu) for _ in range(self.hidden_size)]
    
    def __call__(self, data):
        return np.array([n(data) for n in self.neurons])

In [8]:
l = Layer(10)
l.initialize()
l.neurons

In [9]:
l.neurons[0].weight

-2.4774667335930642

In [10]:
out = l(df.iloc[0, :-1]) # 4 -> 10
out

array([0.        , 0.        , 0.42906082, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [11]:
np.expand_dims(out, axis=1).shape

(10, 1)

In [12]:
np.random.normal(size=(10,3)).shape

(10, 3)

In [13]:
weights = np.random.normal(size=(10,3)) 
out /= np.linalg.norm(out)
weights /= np.linalg.norm(weights)

logits = np.expand_dims(out, axis=1).T @ weights
logits

array([[-0.01686213,  0.11399819, -0.22595957]])

In [14]:
from scipy.special import softmax

In [15]:
probs = softmax(logits)

In [16]:
probs

array([[0.33885386, 0.38622853, 0.27491762]])

In [17]:
df.iloc[0, -1]

np.float64(0.0)

In [ ]:
def make_one_hot(label):
    one_hot = np.zeros(3)
    one_hot[int(label)] = 1
    return one_hot

In [19]:
make_one_hot(df.iloc[0, -1])

array([1., 0., 0.])

In [20]:
from sklearn.metrics import log_loss

In [21]:
log_loss(make_one_hot(df.iloc[0, -1]), probs.squeeze())

0.6305963273243558

# just layers

In [26]:
hidden_size = 10
input = np.expand_dims(df.iloc[0, :-1].to_numpy(), axis=1)
input /= np.linalg.norm(input)

layer1 = np.random.normal(size=(hidden_size, 4))
layer1 /= np.linalg.norm(layer1)

input.shape, layer1.shape

((4, 1), (10, 4))

In [27]:
act = relu(layer1 @ input)
act

array([[0.01652762],
       [0.01252089],
       [0.10072518],
       [0.01539276],
       [0.09080311],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ]])

In [29]:
act.shape

(10, 1)

In [34]:
layer2 = np.random.normal(size=(hidden_size,3)) 
layer2 /= np.linalg.norm(layer2)

act /= np.linalg.norm(act)

output = (layer2.T @ act).squeeze()
output

array([-0.04089742,  0.15918066,  0.13058024])

In [41]:
preds = softmax(output)
trues = make_one_hot(df.iloc[0, -1])
preds.shape, trues.shape

((3,), (3,))

In [50]:
loss = cross_entropy_loss(trues, preds)
loss

np.float64(1.2262886644256403)

In [52]:
d_output = act
d_act = input

In [64]:
lr = .1
layer2 -= lr * d_output
layer1 -= lr * d_act.T

In [65]:
layer1 /= np.linalg.norm(layer1)
act = relu(layer1 @ input)
layer2 /= np.linalg.norm(layer2)
act /= np.linalg.norm(act)
output = (layer2.T @ act).squeeze()
output

array([-0.10187419, -0.10626312, -0.12084282])

In [66]:
preds = softmax(output)
trues = make_one_hot(df.iloc[0, -1])
loss = cross_entropy_loss(trues, preds)
loss

np.float64(1.0908592547307117)